In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


# Overview
this is for making bentmark of model. I learned a lot from below notebook.

test a lot.

[ICR | Simple EDA & Baseline](https://www.kaggle.com/code/datafan07/icr-simple-eda-baseline)

In [2]:
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')

In [3]:
train.columns = train.columns.str.strip()
test.columns = test.columns.str.strip()

In [4]:
num_cols = train.columns.tolist()[1:-1]
cat_cols = 'EJ'
num_cols.remove(cat_cols)

X = train[num_cols]
y = train['Class']

In [14]:
print(y)

0      1
1      0
2      0
3      0
4      1
      ..
612    0
613    0
614    0
615    0
616    0
Name: Class, Length: 617, dtype: int64


In [15]:
print(X)

           AB          AF          AH          AM         AR        AX  \
0    0.209377  3109.03329   85.200147   22.394407   8.138688  0.699861   
1    0.145282   978.76416   85.200147   36.968889   8.138688  3.632190   
2    0.470030  2635.10654   85.200147   32.360553   8.138688  6.732840   
3    0.252107  3819.65177  120.201618   77.112203   8.138688  3.685344   
4    0.380297  3733.04844   85.200147   14.103738   8.138688  3.942255   
..        ...         ...         ...         ...        ...       ...   
612  0.149555  3130.05946  123.763599    9.513984  13.020852  3.499305   
613  0.435846  5462.03438   85.200147   46.551007  15.973224  5.979825   
614  0.427300  2459.10720  130.138587   55.355778  10.005552  8.070549   
615  0.363205  1263.53524   85.200147   23.685856   8.138688  7.981959   
616  0.482849  2672.53426  546.663930  112.006102   8.138688  3.198099   

           AY         AZ          BC          BD  ...         FI         FL  \
0    0.025578   9.812214    5.55

In [5]:
import catboost as cb

params={
    'iterations':10000,
    'learning_rate':0.005,
    'early_stopping_rounds':1000,
    'auto_class_weights':'Balanced',
    'loss_function':'MultiClass',
    'eval_metric':'MultiClass:use_weights=True',
    'random_seed':42,
    'use_best_model':True,
    'l2_leaf_reg':1,
    'max_ctr_complexity':15,
    'max_depth':10,
    "grow_policy":'Lossguide',
    'max_leaves':64,
    "min_data_in_leaf":40,
    }

model = cb.CatBoostClassifier(**params)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
from sklearn.metrics import log_loss
def balance_loglossv2(y_true, y_pred):
    target_mean = y_true.mean()
    w0 = 1/(1-target_mean)
    w1 = 1/target_mean
    sample_weight = [w0 if y == 0 else w1 for y in y_true]
    loss = log_loss(y_true, y_pred, sample_weight=sample_weight)
    return loss


from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [22]:
final_preds = []
for train_index, val_index in kf.split(X):
    
    X_train, X_val = X.iloc[train_index, :], X.iloc[val_index, :]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    model.fit(X_train,y_train,eval_set=(X_val,y_val), verbose=1000)
    y_pred = model.predict(X_val)
    score = balance_loglossv2(y_val, y_pred)
    final_preds.append(score)

0:	learn: 0.6907904	test: 0.6917107	best: 0.6917107 (0)	total: 12.2ms	remaining: 2m 2s
1000:	learn: 0.1013152	test: 0.3053995	best: 0.3053995 (1000)	total: 10.6s	remaining: 1m 34s
2000:	learn: 0.0233281	test: 0.2871847	best: 0.2841594 (1649)	total: 20.9s	remaining: 1m 23s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.2841593753
bestIteration = 1649

Shrink model to first 1650 iterations.
0:	learn: 0.6911387	test: 0.6917670	best: 0.6917670 (0)	total: 12.9ms	remaining: 2m 9s
1000:	learn: 0.0985794	test: 0.3449778	best: 0.3449211 (975)	total: 10.6s	remaining: 1m 35s
2000:	learn: 0.0221595	test: 0.3688700	best: 0.3435038 (1100)	total: 21.2s	remaining: 1m 24s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.3435037725
bestIteration = 1100

Shrink model to first 1101 iterations.
0:	learn: 0.6910158	test: 0.6915881	best: 0.6915881 (0)	total: 9.37ms	remaining: 1m 33s
1000:	learn: 0.1026365	test: 0.3379691	best: 0.3378867 (999)	total: 10.6s	remaining

In [24]:
mean_score = np.mean(final_preds)
print(mean_score)

4.135772877567461


In [25]:
print(final_preds)

[4.533792879134234, 4.513487790704955, 5.628779354646108, 2.8982039663902364, 3.104600396961771]


In [26]:
pridict = model.predict_proba(X)

In [28]:
print(pridict)

[[1.87361591e-03 9.98126384e-01]
 [9.99077953e-01 9.22046581e-04]
 [9.91102463e-01 8.89753711e-03]
 ...
 [9.98117242e-01 1.88275821e-03]
 [9.98216418e-01 1.78358199e-03]
 [9.95195711e-01 4.80428944e-03]]


In [32]:
predict_proba = model.predict_proba(X)

In [33]:
print(predict_proba)

[[1.87361591e-03 9.98126384e-01]
 [9.99077953e-01 9.22046581e-04]
 [9.91102463e-01 8.89753711e-03]
 ...
 [9.98117242e-01 1.88275821e-03]
 [9.98216418e-01 1.78358199e-03]
 [9.95195711e-01 4.80428944e-03]]
